# Module 6: VacationPy

### Importing needed modules

In [ ]:
import pandas as pd
import gmaps
import requests

from config import g_key

### Reading CSV derived from WeatherPy

In [2]:
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Nikolskoye,RU,2020-06-25 23:03:33,59.70,30.79,66.20,72,0,4.47
1,1,Pacific Grove,US,2020-06-25 23:02:00,36.62,-121.92,69.01,68,40,14.99
2,2,Punta Arenas,CL,2020-06-25 23:03:33,-53.15,-70.92,30.20,92,11,5.82
3,3,Sitka,US,2020-06-25 23:03:33,57.05,-135.33,53.60,66,90,11.41
4,4,Pevek,RU,2020-06-25 23:03:34,69.70,170.31,33.53,94,52,15.14


### Configuring Gmaps API Key

In [14]:
gmaps.configure(api_key=g_key)

### Creating a Temperature Heat Layer map using a google maps and previous weatherPy Data

In [26]:
locations = city_data_df[["Lat", "Lng"]]
max_temp = city_data_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Creating a Percent Humidity Heatmap

In [33]:
locations = city_data_df[["Lat","Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
hum_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(hum_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Creating a Percent Cloudiness Heatmap

In [36]:
locations = city_data_df[["Lat",'Lng']]
clouds = city_data_df["Cloudiness"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
cloud_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(cloud_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Creating a Wind Speed Heatmap

In [37]:
locations = city_data_df[["Lat",'Lng']]
wind_s = city_data_df["Wind Speed"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
cloud_layer = gmaps.heatmap_layer(locations, weights=wind_s, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(cloud_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Filtering Cities based on temp inputs from user

In [46]:
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
9,9,Tabou,CI,2020-06-25 23:03:34,4.42,-7.35,76.03,85,95,9.40
10,10,Cockburn Town,TC,2020-06-25 23:03:34,21.46,-71.14,82.89,80,2,19.86
14,14,Puerto Ayora,EC,2020-06-25 23:03:35,-0.74,-90.35,75.20,69,75,24.16
15,15,Saskylakh,RU,2020-06-25 23:03:35,71.92,114.08,82.85,30,54,9.89
18,18,Ibra,OM,2020-06-25 23:03:35,22.69,58.53,78.44,79,32,2.59
19,19,Riyadh,SA,2020-06-25 23:03:04,24.69,46.72,86.00,16,0,4.70
21,21,Fare,PF,2020-06-25 23:03:35,-16.70,-151.02,79.56,80,81,12.44
26,26,San Cristobal,VE,2020-06-25 22:59:05,7.77,-72.22,89.60,43,40,6.93
27,27,Nara,JP,2020-06-25 23:03:36,34.69,135.80,78.80,94,75,5.82
29,29,Kupang,ID,2020-06-25 23:03:36,-10.17,123.58,77.00,78,75,10.29


In [48]:
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Tabou,CI,76.03,4.42,-7.35,
10,Cockburn Town,TC,82.89,21.46,-71.14,
14,Puerto Ayora,EC,75.20,-0.74,-90.35,
15,Saskylakh,RU,82.85,71.92,114.08,
18,Ibra,OM,78.44,22.69,58.53,
19,Riyadh,SA,86.00,24.69,46.72,
21,Fare,PF,79.56,-16.70,-151.02,
26,San Cristobal,VE,89.60,7.77,-72.22,
27,Nara,JP,78.80,34.69,135.80,
29,Kupang,ID,77.00,-10.17,123.58,


### Iterating Through hotel_df DataFrame

In [49]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


### Creating a Maximum Temperature Heatmap from Hotel DataFrame

In [53]:
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

### Creating Info_box Template for allocating info based on HTML Tags, and then creating a list by iterating through hotel_df

In [54]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [55]:
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))